In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tangerino = '/content/drive/MyDrive/RFP-20241111-112510-062e5b.xls'

df = pd.read_excel(tangerino)
pd.set_option('display.max_columns', None)

In [ ]:
#Apagar as 5 primeiras linhas que trazem dados do empregador para não confundir com os dados dos colaboradores no momento da extração em lista
#RODAR APENAS UMA VEZZZZZZZ

df = df.drop(df.index[:5])


In [ ]:
#Cópia dos dados para uma nova coluna -> conversão para data -> conversão para string (manipulação)

#Admissão
df['Data_Admissao_Auxiliar'] = df['Unnamed: 3']
df['Data_Admissao_Auxiliar'] = pd.to_datetime(df['Data_Admissao_Auxiliar'], errors='coerce')
df['Data_Admissao_Auxiliar']=df['Data_Admissao_Auxiliar'].astype(str)

#Ponto
df['Data_Ponto_Auxiliar'] = df['Folha de Ponto']
df['Data_Ponto_Auxiliar'] = pd.to_datetime(df['Data_Ponto_Auxiliar'], errors='coerce')
df['Data_Ponto_Auxiliar']=df['Data_Ponto_Auxiliar'].astype(str)

<ipython-input-4-fbff3fee3487>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Data_Admissao_Auxiliar'] = pd.to_datetime(df['Data_Admissao_Auxiliar'], errors='coerce')
<ipython-input-4-fbff3fee3487>:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Data_Ponto_Auxiliar'] = pd.to_datetime(df['Data_Ponto_Auxiliar'], errors='coerce')


In [ ]:
# Identificar as linhas onde a coluna "X" contém "Chave:" /(formato: df)
nome_ln = df[df['Folha de Ponto'].str.contains("Nome:", na=False)]
cpf_ln = df[df['Unnamed: 21'].str.contains("CPF:", na=False)]
admissao_ln = df[df['Folha de Ponto'].str.contains("Admissão:", na=False)]
cpts_ln = df[df['Unnamed: 7'].str.contains("CTPS:", na=False)]
serie_ln = df[df['Unnamed: 14'].str.contains("Série:", na=False)]
funcao_ln = df[df['Unnamed: 21'].str.contains("Função:", na=False)]

#Extrair valores das colunas de acordo com identificador /(formato: lista)
nomes = nome_ln['Unnamed: 3'].values
cptss = cpts_ln['Unnamed: 9'].values
series = serie_ln['Unnamed: 16'].values
admissoes = admissao_ln['Data_Admissao_Auxiliar'].values
funcoes = funcao_ln['Unnamed: 26'].values
cpfs= cpf_ln['Unnamed: 23'].values

In [ ]:
# Lista de datas para o mês de outubro
datas_outubro = [f"2024-10-{str(i).zfill(2)}" for i in range(1, 32)]

colaboradores = []

min_len = min(len(nomes), len(cpfs), len(cptss), len(series), len(admissoes), len(funcoes))

# Montar o dicionário de cada colaborador /(formato: chave-valor)
for i in range(min_len):
    colaborador = {
        'Nome': nomes[i] if i < len(nomes) else None,
        'CPF': cpfs[i] if i < len(cpfs) else None,
        'CTPS': cptss[i] if i < len(cptss) else None,
        'Série': series[i] if i < len(series) else None,
        'Admissão': admissoes[i] if i < len(admissoes) else None,
        'Função': funcoes[i] if i < len(funcoes) else None,
    }

    #Montar DF linha a linha + lista com valores de acordo com a lista datas_outubro
    for data in datas_outubro:
        pontos_dia_ln = df[(df['Data_Ponto_Auxiliar'].str.contains(data, na=False))]
        observacao_ln = df[(df['Data_Ponto_Auxiliar'].str.contains(data, na=False))]
        if not pontos_dia_ln.empty:
            pontos_dia = pontos_dia_ln['Unnamed: 22'].iloc[i] if i < len(pontos_dia_ln) else None
            observacao = observacao_ln['Unnamed: 10'].iloc[i] if i < len(pontos_dia_ln) else None
        else:
            pontos_dia = None

        # Cria a chave [data] e a atribui no dicionário `colaborador`. Também adiciona o valor da tupla (pontos_dia, observacao) referente à cada chave
        colaborador[data] = pontos_dia, observacao

    #Acrescentar registro na lista
    colaboradores.append(colaborador)

In [ ]:
colaboradores


[{'Nome': 'ADILÉIA ROCHA SANTOS',
  'CPF:': 6693058639.0,
  'CTPS': 6087337.0,
  'Série': '30',
  'Admissão': '2021-11-03',
  'Função:': 'ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR',
  '2024-10-01': (nan, ' COMPENSAÇÃO DE HORAS |  COMPENSAÇÃO DE HORAS | '),
  '2024-10-02': (nan, ' COMPENSAÇÃO DE HORAS |  COMPENSAÇÃO DE HORAS | '),
  '2024-10-03': (nan, ' COMPENSAÇÃO DE HORAS |  COMPENSAÇÃO DE HORAS | '),
  '2024-10-04': (nan, ' COMPENSAÇÃO DE HORAS |  COMPENSAÇÃO DE HORAS | '),
  '2024-10-05': (nan, ' - '),
  '2024-10-06': (nan, ' - '),
  '2024-10-07': ('09:12', '07:56 12:22 | (m)13:23 18:09 | '),
  '2024-10-08': ('08:35', '12:22 17:22 | (m)18:22 21:57 | '),
  '2024-10-09': ('10:32', '(m)08:51 14:50 | (m)15:50 20:23 | '),
  '2024-10-10': ('13:16',
   '(m)09:02 14:01 | (m)15:00 21:00 | (m)21:20 23:37 | '),
  '2024-10-11': ('09:05', '(m)09:12 13:02 | (m)14:03 19:18 | '),
  '2024-10-12': (nan, 'FERIADO: Nossa Senhora Aparecida'),
  '2024-10-13': (nan, ' - '),
  '2024-10-14': ('09:11', '

In [ ]:
# Inicializa uma lista para armazenar os dados
dados = []

# Itera sobre cada colaborador na lista
for colaborador in colaboradores:
    # Extrai os dados principais (nome, cpf, ctps, serie)
    nome = colaborador['Nome']
    cpf = colaborador['CPF']
    ctps = colaborador['CTPS']
    serie = colaborador['Série']
    admissao = colaborador['Admissão']
    funcao = colaborador['Função']

    # Itera sobre os registros de chave
    for chave_data, tupla in colaborador.items():
        if chave_data not in ['Nome', 'CPF', 'CTPS', 'Série', 'Admissão', 'Função']:
            hora, observacao = tupla
            # Cria um dicionário com os dados para essa chave e adiciona à lista
            dados.append({
                'nome': nome,
                'cpf': cpf,
                'ctps': ctps,
                'serie': serie,
                'admissao': admissao,
                'funcao': funcao,
                'data': chave_data,
                'hora': hora if pd.notna(hora) else '',
                'observacao': observacao if pd.notna(observacao) else ''
            })

# Cria o DataFrame
df_final = pd.DataFrame(dados)

# Exibe o DataFrame
df_final.head(40)


,nome,cpf,ctps,serie,admissao,funcao,data,hora,observacao
0,ADILÉIA ROCHA SANTOS,6.693059e+09,6087337.0,30,2021-11-03,ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR,2024-10-01,,COMPENSAÇÃO DE HORAS | COMPENSAÇÃO DE HORAS |
1,ADILÉIA ROCHA SANTOS,6.693059e+09,6087337.0,30,2021-11-03,ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR,2024-10-02,,COMPENSAÇÃO DE HORAS | COMPENSAÇÃO DE HORAS |
2,ADILÉIA ROCHA SANTOS,6.693059e+09,6087337.0,30,2021-11-03,ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR,2024-10-03,,COMPENSAÇÃO DE HORAS | COMPENSAÇÃO DE HORAS |
3,ADILÉIA ROCHA SANTOS,6.693059e+09,6087337.0,30,2021-11-03,ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR,2024-10-04,,COMPENSAÇÃO DE HORAS | COMPENSAÇÃO DE HORAS |
4,ADILÉIA ROCHA SANTOS,6.693059e+09,6087337.0,30,2021-11-03,ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR,2024-10-05,,-
5,ADILÉIA ROCHA SANTOS,6.693059e+09,6087337.0,30,2021-11-03,ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR,2024-10-06,,-
6,ADILÉIA ROCHA SANTOS,6.693059e+09,6087337.0,30,2021-11-03,ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR,2024-10-07,09:12,07:56 12:22 | (m)13:23 18:09 |
7,ADILÉIA ROCHA SANTOS,6.693059e+09,6087337.0,30,2021-11-03,ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR,2024-10-08,08:35,12:22 17:22 | (m)18:22 21:57 |
8,ADILÉIA ROCHA SANTOS,6.693059e+09,6087337.0,30,2021-11-03,ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR,2024-10-09,10:32,(m)08:51 14:50 | (m)15:50 20:23 |
9,ADILÉIA ROCHA SANTOS,6.693059e+09,6087337.0,30,2021-11-03,ASSESSOR(A) TÉCNICO(A) NÍVEL SUPERIOR JUNIOR,2024-10-10,13:16,(m)09:02 14:01 | (m)15:00 21:00 | (m)21:20 23:...


In [ ]:
# Salvar DF no drive

nome_arquivo = 'df_final.xlsx'
caminho_arquivo = f'/content/drive/MyDrive/{nome_arquivo}'

df_final.to_excel(caminho_arquivo, index=False)

Arquivo Excel salvo em: /content/drive/MyDrive/df_final.xlsx
